# Actor-Critic

Теорема о градиенте стратегии связывает градиент целевой функции  и градиент самой стратегии:

$$\nabla_\theta J(\theta) = \mathbb{E}_\pi [Q^\pi(s, a) \nabla_\theta \ln \pi_\theta(a \vert s)]$$

Встает вопрос, как оценить $Q^\pi(s, a)$? В чистом policy-based алгоритме REINFORCE используется отдача $G_t$, полученная методом Монте-Карло в качестве несмещенной оценки $Q^\pi(s, a)$. В Actor-Critic же предлагается отдельно обучать нейронную сеть Q-функции — критика.

Актор-критиком часто называют обобщенный фреймворк (подход), нежели какой-то конкретный алгоритм. Как подход актор-критик не указывает, каким конкретно [policy gradient] методом обучается актор и каким [value based] методом обучается критик. Таким образом актор-критик задает целое [семейство](https://proceedings.neurips.cc/paper_files/paper/1999/file/6449f44a102fde848669bdd9eb6b76fa-Paper.pdf) различных алгоритмов. Рекомендую в качестве шпаргалки использовать упомянутый в тетрадке с REINFORCE [пост из блога Lilian Weng](https://lilianweng.github.io/posts/2018-04-08-policy-gradient/), посвященный наиболее популярным алгоритмам семейства актор-критиков

В данной тетрадке познакомимся с наиболее простым вариантом актор-критика, который так и называют Actor-Critic:

In [15]:
# Cтавим нужные зависимости, если это колаб
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [16]:
from collections import deque

import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.distributions import Categorical

%matplotlib inline

In [17]:
env = gym.make("CartPole-v1")
env.reset()

print(f'{env.observation_space=}')
print(f'{env.action_space=}')

n_actions = env.action_space.n
state_dim = env.observation_space.shape
print(f'Action_space: {n_actions} | State_space: {env.observation_space.shape}')

env.observation_space=Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
env.action_space=Discrete(2)
Action_space: 2 | State_space: (4,)


(1 балл)

In [18]:
def to_tensor(x, dtype=np.float32):
    if isinstance(x, torch.Tensor):
        return x
    x = np.asarray(x, dtype=dtype)
    x = torch.from_numpy(x)
    return x

def symlog(x):
    """Compute symlog values for a vector `x`. It's an inverse operation for symexp."""
    return torch.sign(x) * torch.log(torch.abs(x) + 1)

def symexp(x):
    """Compute symexp values for a vector `x`. It's an inverse operation for symlog."""
    return torch.sign(x) * (torch.exp(torch.abs(x)) - 1.0)


class SymExpModule(nn.Module):
    def forward(self, x):
        return symexp(x)

def select_action_eps_greedy(Q, state, epsilon):
    """Выбирает действие epsilon-жадно."""
    if not isinstance(state, torch.Tensor):
        state = torch.tensor(state, dtype=torch.float32)
    Q_s = Q(state).detach().numpy()

    if np.random.random() < epsilon:
        action = np.random.randint(len(Q_s))
    else:
        action = np.argmax(Q_s)

    action = int(action)
    return action

def sample_batch(replay_buffer, n_samples):
    # sample randomly `n_samples` samples from replay buffer
    # and split an array of samples into arrays: states, actions, rewards, next_actions, terminateds
    if len(replay_buffer) < n_samples:
        n_samples = len(replay_buffer)

    indices = np.random.choice(len(replay_buffer), n_samples, replace=False)
    batch = [replay_buffer[i] for i in indices]

    states = [experience[0] for experience in batch]
    actions = [experience[1] for experience in batch]
    rewards = [experience[2] for experience in batch]
    next_states = [experience[3] for experience in batch]
    terminateds = [experience[4] for experience in batch]

    return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(terminateds)

## Shared-body Actor-Critic

Актор и критик могут обучаться в разных режимах — актор только on-policy (шаг обучения на текущей собранной подтраектории), а критик on-policy или off-policy (шаг обучения на текущей подтраектории или на батче из replay buffer). Это с одной стороны привносит гибкость в обучение, с другой — усложняет его.

Если актор и критик оба обучаются on-policy, то имеет смысл объединить их сетки в одну и делать общий шаг обратного распространения ошибки. Однако, если они обучаются в разных режимах (и с разной частотой обновления), то велика вероятность, что их шаги обучения могут начать конфликтовать в случае общего тела — для такого варианта намного предпочтительнее разделить их на разные подсети (либо аккуратно настраивать гиперпарметры, чтобы стабилизировать обучение). В целом, рекомендуется использовать общий энкодер наблюдений, а далее как можно скорее разделять головы.

Сделаем реализацию актор-критика с общим телом и с on-policy вариантом обучения.

In [19]:
class ActorBatch:
    def __init__(self):
        self.logprobs = []
        self.q_values = []

    def append(self, log_prob, q_value):
        self.logprobs.append(log_prob)
        self.q_values.append(q_value)

    def clear(self):
        self.logprobs.clear()
        self.q_values.clear()

(3 балла)

In [21]:
class ActorCriticModel(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super().__init__()
        self.output_dim = output_dim

        # Исправляем обработку input_dim
        if isinstance(input_dim, tuple):
            input_size = input_dim[0]
        else:
            input_size = input_dim

        layers = []
        prev_dim = input_size
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            prev_dim = hidden_dim

        self.net = nn.Sequential(*layers)

        self.actor_head = nn.Sequential(
            nn.Linear(prev_dim, output_dim),
            nn.Softmax(dim=-1)
        )

        self.critic_head = nn.Linear(prev_dim, output_dim)

    def forward(self, state):
        if not isinstance(state, torch.Tensor):
            state = torch.FloatTensor(state)

        # Обрабатываем как одиночное состояние, так и батч
        if len(state.shape) == 1:
            state = state.unsqueeze(0)

        features = self.net(state)
        action_probs = self.actor_head(features)

        dist = Categorical(action_probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)

        q_values = self.critic_head(features)
        Q_s_a = q_values.gather(1, action.unsqueeze(-1)).squeeze(-1)

        return action, log_prob, Q_s_a

    def evaluate(self, state):
        if not isinstance(state, torch.Tensor):
            state = torch.FloatTensor(state)

        if len(state.shape) == 1:
            state = state.unsqueeze(0)

        features = self.net(state)
        q_values = self.critic_head(features)
        return q_values

(6 баллов)

In [24]:
class ActorCriticAgent:
    def __init__(self, state_dim, action_dim, hidden_dims, lr, gamma, critic_rb_size):
        self.lr = lr
        self.gamma = gamma

        self.actor_critic = ActorCriticModel(state_dim, hidden_dims, action_dim)
        self.opt = torch.optim.Adam(self.actor_critic.parameters(), lr=lr)

        self.actor_batch = ActorBatch()
        self.critic_rb = deque(maxlen=critic_rb_size)

    def act(self, state):
        # УБИРАЕМ torch.no_grad() - нам нужны градиенты для log_prob
        action, log_prob, q_value = self.actor_critic(state)

        # q_value детачим, но log_prob оставляем с градиентами
        self.actor_batch.append(log_prob, q_value.detach())

        return action.item()

    def append_to_replay_buffer(self, s, a, r, next_s, terminated):
        self.critic_rb.append((s, a, r, next_s, terminated))

    def evaluate(self, state):
        return self.actor_critic.evaluate(state)

    def update(self, rollout_size, critic_batch_size, critic_updates_per_actor):
        if len(self.actor_batch.q_values) < rollout_size:
            return

        self.opt.zero_grad()
        critic_loss = self.update_critic(critic_batch_size, critic_updates_per_actor)
        actor_loss = self.update_actor()
        total_loss = critic_loss + actor_loss

        # Проверяем, требует ли loss градиенты
        if not total_loss.requires_grad:
            print("Warning: total_loss doesn't require grad!")
            return

        total_loss.backward()
        self.opt.step()
        self.actor_batch.clear()

    def update_actor(self):
        Q_s_a = torch.stack(self.actor_batch.q_values)
        logprobs = torch.stack(self.actor_batch.logprobs)

        # Проверяем, что logprobs требуют градиенты
        if not logprobs.requires_grad:
            # Если нет, пересчитываем с градиентами
            print("Warning: logprobs don't require grad, recalculating...")
            states = [exp[0] for exp in self.critic_rb][-len(logprobs):]
            states_tensor = torch.FloatTensor(np.array(states))
            _, new_logprobs, _ = self.actor_critic(states_tensor)
            logprobs = torch.stack(new_logprobs)

        # Policy gradient loss
        loss = -torch.mean(Q_s_a * logprobs)
        return loss

    def update_critic(self, batch_size, n_updates=1):
        if len(self.critic_rb) < batch_size:
            # Возвращаем тензор, который требует градиенты
            return torch.tensor(0.0, requires_grad=True)

        total_loss = torch.tensor(0.0, requires_grad=True)
        for _ in range(n_updates):
            states, actions, rewards, next_states, terminateds = sample_batch(
                self.critic_rb, batch_size
            )

            loss = self.compute_td_loss(states, actions, rewards, next_states, terminateds)
            total_loss = total_loss + loss

        return total_loss / n_updates

    def compute_td_loss(self, states, actions, rewards, next_states, terminateds, regularizer=0.1):
        s = to_tensor(states)
        a = to_tensor(actions, int).long()
        r = to_tensor(rewards)
        s_next = to_tensor(next_states)
        term = to_tensor(terminateds, bool)

        # Вычисляем Q-значения - они будут требовать градиенты
        q_values = self.actor_critic.evaluate(s)
        Q_s_a = q_values.gather(1, a.unsqueeze(1)).squeeze(1)

        # Целевые значения вычисляем без градиентов
        with torch.no_grad():
            q_next = self.actor_critic.evaluate(s_next)
            V_sn, _ = torch.max(q_next, dim=1)

        target = r + self.gamma * V_sn * (~term).float()
        td_error = target - Q_s_a

        loss = torch.mean(td_error ** 2)
        loss += regularizer * torch.mean(Q_s_a ** 2)
        return loss

In [25]:
def run_actor_critic(
        env_name="CartPole-v1",
        hidden_dims=(128, 128), lr=5e-4,
        total_max_steps=200_000,
        train_schedule=16, replay_buffer_size=50000, batch_size=64, critic_updates_per_actor=4,
        eval_schedule=1000, smooth_ret_window=10, success_ret=200.
):
    env = gym.make(env_name)
    episode_return_history = deque(maxlen=smooth_ret_window)

    agent = ActorCriticAgent(
        state_dim=env.observation_space.shape[0], action_dim=env.action_space.n, hidden_dims=hidden_dims,
        lr=lr, gamma=.995, critic_rb_size=replay_buffer_size
    )

    s, _ = env.reset()
    done, episode_return = False, 0.
    eval = False

    for global_step in range(1, total_max_steps+1):
        a = agent.act(s)
        s_next, r, terminated, truncated, _ = env.step(a)
        episode_return += r
        done = terminated or truncated

        # train step
        agent.append_to_replay_buffer(s, a, r, s_next, terminated)
        agent.update(train_schedule, batch_size, critic_updates_per_actor)

        # evaluate
        if global_step % eval_schedule == 0:
            eval = True

        s = s_next
        if done:
            if eval:
                episode_return_history.append(episode_return)
                avg_return = np.mean(episode_return_history)
                print(f'{global_step=} | {avg_return=:.3f}')
                if avg_return >= success_ret:
                    print('Решено!')
                    break

            s, _ = env.reset()
            done, episode_return = False, 0.
            eval = False

run_actor_critic(eval_schedule=2000, total_max_steps=100_000)

global_step=2002 | avg_return=10.000
global_step=4002 | avg_return=10.500
global_step=6019 | avg_return=18.667
global_step=8023 | avg_return=20.000
global_step=10013 | avg_return=22.400
global_step=12000 | avg_return=24.833
global_step=14042 | avg_return=29.857
global_step=16036 | avg_return=37.500
global_step=18068 | avg_return=49.889
global_step=20000 | avg_return=64.100
global_step=22189 | avg_return=84.500
global_step=24037 | avg_return=99.100
global_step=26125 | avg_return=126.300
global_step=28077 | avg_return=133.300
global_step=30100 | avg_return=147.500
global_step=32052 | avg_return=150.500
global_step=34042 | avg_return=167.400
global_step=36168 | avg_return=181.700
global_step=38013 | avg_return=188.100
global_step=40053 | avg_return=182.500
global_step=42026 | avg_return=186.300
global_step=44033 | avg_return=186.000
global_step=46004 | avg_return=173.200
global_step=48205 | avg_return=191.000
global_step=50168 | avg_return=193.300
global_step=52177 | avg_return=205.300
Ре